In [1]:
import bioread
import numpy as np
import pandas as pd
import neurokit2 as nk
import matplotlib.pyplot as plt

In [2]:
class Segment(object):
    def __init__(self, name, start_index_text, start_index, seg_before):
        self.name = name
        self.start_index_text = start_index_text
        self.start_index = start_index
        self.end_index_text = ""
        self.end_index = 0
        self.before = seg_before
        self.after = None
        self.marker_inside_text = ""
        self.marker_inside_index = 0

    def make_df(self, file):
        data = {}
        for channel in file.named_channels:
            signal = np.array(file.named_channels[channel].data[self.start_index:self.end_index])
            data[channel] = signal
        self.df = pd.DataFrame(data)
        print('Dataframe of {0}:'.format(self.name))
        #print(self.df)


In [3]:
class Filedata(object):
    def __init__(self, filename):
        self.filename = filename
        self.segments = {}

    def preparedata(self):
        name_seg_before = None
        name_seg_after = None
        file = bioread.read_file('../' + self.filename)
        for m in file.event_markers:
            if self.filename == 'RAW_data/PB13.acq':
                if m.text == 'end 2D NF ':
                   m.text = 'end 2D NF'
                if m.text == 'Segment 1':
                    name = m.text
                    current_seg = Segment(name, m.text, m.sample_index, name_seg_before)
                    self.segments[name] = current_seg
                    name_seg_before = name
            if self.filename == 'RAW_data/PB12.acq':
                if m.text == 'ABBA end':
                   m.text = 'end ABBA'
                if m.text == 'end VR NF':
                   m.text = 'end NF VR'
                if m.text == 'Segment 3':
                   m.text = 'end MIST'
                if m.text == 'Segment 1' or m.text == 'Segment 2':
                    name = m.text
                    current_seg = Segment(name, m.text, m.sample_index, name_seg_before)
                    self.segments[name] = current_seg
                    name_seg_before = name
            if self.filename == 'RAW_data/PB5.acq':
                if m.text == 'end NF VR':
                   m.text = 'end VR NF'
                if m.text == 'Segment 1' or m.text == 'Segment 2':
                    name = m.text
                    current_seg = Segment(name, m.text, m.sample_index, name_seg_before)
                    self.segments[name] = current_seg
                    name_seg_before = name
            if self.filename == 'RAW_data/PB17.acq':
                if m.text == 'end 2D NF' and m.sample_index == 4400451:
                   m.text = 'end NF-VR'
                if m.text == 'Segment 1':
                    name = m.text
                    current_seg = Segment(name, m.text, m.sample_index, name_seg_before)
                    self.segments[name] = current_seg
                    name_seg_before = name
            if   m.text.startswith("start "):
                name = m.text.removeprefix("start ")
                current_seg = Segment(name, m.text, m.sample_index, name_seg_before)
                self.segments[name] = current_seg
                if name_seg_before != None:
                    current_seg.before = name_seg_before
                    self.segments[name_seg_before].after = name
                name_seg_before = name
            elif m.text.startswith("end "):
                name = m.text.removeprefix("end ")
                current_seg = self.segments[name]
                current_seg.end_index_text = m.text
                current_seg.end_index = m.sample_index
                current_seg.make_df(file)
                name_seg_before = name
            else:
                if name_seg_before != None:
                    print("insider: " + m.text)
                    self.segments[name_seg_before].marker_inside_text = m.text
                    self.segments[name_seg_before].marker_inside_index = m.sample_index
            print(m.text)

        # Out of the event_markers loop
        if self.filename == 'RAW_data/PB17.acq':
            current_seg = self.segments['Segment 1']
            current_seg.end_index_text = self.segments['ABBA'].start_index_text
            current_seg.end_index = self.segments['ABBA'].start_index
            current_seg.make_df(file)
        if self.filename == 'RAW_data/PB5.acq':
            current_seg = self.segments['Segment 1']
            current_seg.end_index_text = self.segments['Segment 2'].start_index_text
            current_seg.end_index = self.segments['Segment 2'].start_index
            current_seg.make_df(file)
            current_seg = self.segments['Segment 2']
            current_seg.end_index_text = self.segments['MIST'].start_index_text
            current_seg.end_index = self.segments['MIST'].start_index
            current_seg.make_df(file)
        if self.filename == 'RAW_data/PB12.acq':
            current_seg = self.segments['Segment 1']
            current_seg.end_index_text = self.segments['Segment 2'].start_index_text
            current_seg.end_index = self.segments['Segment 2'].start_index
            current_seg.make_df(file)
            current_seg = self.segments['Segment 2']
            current_seg.end_index_text = self.segments['ABBA'].start_index_text
            current_seg.end_index = self.segments['ABBA'].start_index
            current_seg.make_df(file)
        if self.filename == 'RAW_data/PB13.acq':
            current_seg = self.segments['Segment 1']
            current_seg.end_index_text = self.segments['ABBA'].start_index_text
            current_seg.end_index = self.segments['ABBA'].start_index
            current_seg.make_df(file)

In [4]:
def ECG_report(df, name):
    print('Segment {0}:'.format(name))
    nk.signal_plot(df, subplots=True, sampling_rate=1000)
    #fig = plt.gcf()
    #fig.savefig("all_{0}.png".format(name))
    plt.close()

    print('ECG of segment {0}:'.format(name))
    nk.signal_plot(df['ECG (.5 - 35 Hz)'], sampling_rate=1000)
    #fig = plt.gcf()
    #fig.savefig("ecg_{0}.png".format(name))
    plt.close()
    # Find peaks
    peaks, info = nk.ecg_peaks(df['ECG (.5 - 35 Hz)'], sampling_rate=1000)
    # Compute HRV indices
    hrv = nk.hrv_time(peaks, sampling_rate=1000, show=True)
    #fig = plt.gcf()
    #fig.savefig("hrv_{0}.png".format(name))
    plt.close()
    print('HRV of segment {0}:'.format(name))
    print(hrv)
    #for col in hrv:
    #    print(col)
    print(hrv['HRV_MeanNN'])
    #print(hrv.iloc[0]['HRV_MeanNN'])
    print(hrv['HRV_SDNN'])
    #print(hrv.iloc[0]['HRV_SDNN'])
    print(hrv['HRV_RMSSD'])
    #print(hrv.iloc[0]['HRV_RMSSD'])

In [5]:
def EDA_report(df, name):
    #reportname = 'EDAreport_{0}.html'.format(name)
    #signals, info = nk.eda_process(df['EDA (0 - 35 Hz)'], sampling_rate=1000, report=reportname)
    #signals, info = nk.eda_process(df['EDA (0 - 35 Hz)'], sampling_rate=1000, report="text")
    signals, info = nk.eda_process(df['EDA (0 - 35 Hz)'], sampling_rate=1000)
    nk.eda_plot(signals, info)
    #fig = plt.gcf()
    #fig.savefig("eda_{0}.png".format(name))
    plt.close()
    print('EDA of segment {0}:'.format(name))
    #print(signals)
    #print(info)
    analyze_df = nk.eda_analyze(signals, sampling_rate=1000)
    print(analyze_df)



In [6]:
def main():
    fdatas = {}
    for filename in ['RAW_data/PB5.acq', 'RAW_data/PB12.acq', 'RAW_data/PB13.acq', 'RAW_data/PB17.acq']:
        fdata = Filedata(filename)
        fdata.preparedata()
        fdatas[filename] = fdata

    for fnam, fdat in fdatas.items():
        print('\nFile {0}:'.format(fnam))
        for name, seg in fdat.segments.items():
            ECG_report(seg.df, name)
            EDA_report(seg.df, name)
            #pass

        print('\nFor file {0}:'.format(fnam))
        for name, seg in fdat.segments.items():
            if seg.marker_inside_index != 0:
                print(seg.marker_inside_index)
                print('segment: [{0}] before: [{1}] after [{2}] start [{3}] end [{4}] marker_inside label[{5}] marker_inside index [{6}]'.format(name, seg.before, seg.after, seg.start_index, seg.end_index, seg.marker_inside_text, seg.marker_inside_index))
            else:
                print('segment: [{0}] before: [{1}] after [{2}] start [{3}] end [{4}]'.format(name, seg.before, seg.after, seg.start_index, seg.end_index))
        #first = ""
        #for name, seg in segments.items():
        #    if seg.before == None:
        #        first = name
        #        break
        #seg = segments[first]
        #while True:
        #    print(seg.name)
        #    if seg.after == None:
        #        break
        #    seg = segments[seg.after]
        #In Python 3.6+ dictionaries preserve insertion order.
        #sys.exit(0)
    return 0

In [7]:
main()

insider: Segment 1
Segment 1
insider: Segment 2
Segment 2
start MIST
Dataframe of MIST:
end MIST
start 2D NF
insider: last block
last block
Dataframe of 2D NF:
end 2D NF
start ABBA
Dataframe of ABBA:
end ABBA
start VR NF
insider: last round
last round
Dataframe of VR NF:
end VR NF
Dataframe of Segment 1:
Dataframe of Segment 2:
insider: Segment 1
Segment 1
insider: Segment 2
Segment 2
start ABBA
Dataframe of ABBA:
end ABBA
start NF VR
insider: last block
last block
Dataframe of NF VR:
end NF VR
start MIST
Dataframe of MIST:
end MIST
Dataframe of Segment 1:
Dataframe of Segment 2:
insider: Segment 1
Segment 1
start ABBA
Dataframe of ABBA:
end ABBA
start 2D NF
Dataframe of 2D NF:
end 2D NF
start MIST
Dataframe of MIST:
end MIST
start VR NF
insider: last block
last block
Dataframe of VR NF:
end VR NF
Dataframe of Segment 1:
insider: Segment 1
Segment 1
start ABBA
Dataframe of ABBA:
end ABBA
insider: 

start 2D NF
insider: last block
last block
Dataframe of 2D NF:
end 2D NF
start MIST
Data

0